In [1]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from utils import toTorchTensers, NormalizeData

data_url = r'./data/Data.xlsx'
data_egypt = pd.read_excel(data_url, sheet_name='Egypt')
data_vietnam = pd.read_excel(data_url, sheet_name='Vietnam')


In [2]:
# data
DATA = data_egypt
DATA = NormalizeData(DATA)
DATA = DATA.to_numpy()

np.random.seed(1)
np.random.shuffle(DATA)
split_point = math.floor(len(DATA) * 0.75)

train_x, train_y = DATA[:split_point, 1:], DATA[:split_point, 0:1]
test_x, test_y = DATA[split_point:, 1:], DATA[split_point:, 0:1]

[train_x, train_y, test_x, test_y] = toTorchTensers(
  train_x, train_y, test_x, test_y
)


In [6]:
# model
class ForcastModel(nn.Module):
  def __init__(self):
    super().__init__()

    self.opti = None
    self.crit = nn.MSELoss()
    self.pipeline = nn.Sequential(
      nn.Linear(12, 48),
      nn.ReLU(),
      nn.Linear(48, 48),
      nn.ReLU(),
      nn.Linear(48, 24),
      nn.ReLU(),
      nn.Linear(24, 24),
      nn.ReLU(),
      nn.Linear(24, 24),
      nn.ReLU(),
      nn.Linear(24, 1),
      nn.Sigmoid()
    )

  def forward(self, x):
    return self.pipeline(x)
  
  def fit(self, data_x, data_y, epoches=100, batch_size=64):
    if(not self.opti):
      self.opti = optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)
      
    for epoche in range(epoches):
      epoche_start_time = timer()

      for i in range(len(data_x)):
        recon = self(data_x[i])
        loss = self.crit(recon, data_y[i])

        self.opti.zero_grad()
        loss.backward()
        self.opti.step()

      epoche_exec_time = f"{(timer() - epoche_start_time):.1f}"
      print(
          f'epoche: {epoche}, loss: {loss.item():.8f}, execution time: {epoche_exec_time}s'
      )
  
  def test(self, data_x, data_y):
    recon = self(data_x[10])
    print(recon)

In [10]:
# training
model = ForcastModel()
model.fit(train_x, train_y, epoches=20)

epoche: 0, loss: 0.00020235, execution time: 1.3s
epoche: 1, loss: 0.00002691, execution time: 1.2s
epoche: 2, loss: 0.00000751, execution time: 1.3s
epoche: 3, loss: 0.00000428, execution time: 1.2s
epoche: 4, loss: 0.00000119, execution time: 1.2s
epoche: 5, loss: 0.00000034, execution time: 1.2s
epoche: 6, loss: 0.00000011, execution time: 1.3s
epoche: 7, loss: 0.00000004, execution time: 1.3s
epoche: 8, loss: 0.00000001, execution time: 1.3s
epoche: 9, loss: 0.00000000, execution time: 1.3s
epoche: 10, loss: 0.00000000, execution time: 1.3s
epoche: 11, loss: 0.00000000, execution time: 1.3s
epoche: 12, loss: 0.00000000, execution time: 1.3s
epoche: 13, loss: 0.00000000, execution time: 1.3s
epoche: 14, loss: 0.00000000, execution time: 1.2s
epoche: 15, loss: 0.00000000, execution time: 1.3s
epoche: 16, loss: 0.00000000, execution time: 1.3s
epoche: 17, loss: 0.00000000, execution time: 1.3s
epoche: 18, loss: 0.00000000, execution time: 1.3s
epoche: 19, loss: 0.00000000, execution t

In [12]:
# testing
model.test(test_x, test_y)


tensor([0.5605], grad_fn=<SigmoidBackward0>)


In [ ]:
for input in test_x:
    recon = model(input)
    loss = crit(recon, input)
    print(f'{i} loss: {loss.item():.4f}')
